In [ ]:
import os
import imageio.v2 as imageio
import torch
from PIL import Image

img_arr = imageio.imread(os.path.join(os.path.pardir, "_00_data", "a_image-dog", "bobby.jpg")) #파일열기

print(type(img_arr))  # 이미지 배열의 데이터 타입 출력
print(img_arr.shape)  # 이미지의 형태 출력
print(img_arr.dtype)  # 이미지의 데이터 타입 출력

img = torch.from_numpy(img_arr) # numpy 배열을 PyTorch 텐서로 변환
out = img.permute(2, 0, 1)  # 이미지의 차원을 (높이, 너비, 채널)에서 (채널, 높이, 너비)로 변환
print(out.shape)  

print("#" * 50, 1)

data_dir = os.path.join(os.path.pardir, "_00_data", "b_image-cats")

# 해당 디렉토리 내에 있는 .png 확장자의 파일명을 가져옴
filenames = [
  name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png'
]
print(filenames)


# 각 파일을 열어서 이미지 출력 및 정보 출력
for i, filename in enumerate(filenames):
  image = Image.open(os.path.join(data_dir, filename))
  image.show()
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  print(img_arr.shape)
  print(img_arr.dtype)

batch_size = 3  # 배치 사이즈 설정 
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8) # (배치, 채널, 높이, 너비)의 빈 텐서 생성


for i, filename in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  batch[i] = img_t  # 변환된 텐서를 배치에 추가

print(batch.shape)  # 배치 텐서의 크기 출력

print("#" * 50, 2)

batch = batch.float()
batch /= 255.0  #이미지값을 0~1로 정규화
print(batch.dtype)
print(batch.shape)

n_channels = batch.shape[1]

for c in range(n_channels):
  mean = torch.mean(batch[:, c])
  std = torch.std(batch[:, c])
  print(mean, std)
  batch[:, c] = (batch[:, c] - mean) / std

# A 고찰내용
#### 이미지 데이터 확인 및 출력
1. 이미지 파일을 텐서로 관리할 수 있다.
2. 텐서 차원의 순서를 변경할 수 있다.
3. 정규화를 할 수 있다.

In [ ]:
import os

import imageio.v2 as imageio

# DICOM 볼륨 데이터가 있는 디렉토리 경로 설정
dir_path = os.path.join(os.path.pardir, "_00_data", "c_volumetric-dicom", "2-LUNG_3.0_B70f-04083")  # 파일 경로 코드 수정
# DICOM 볼륨 데이터를 읽어 numpy 배열로 변환
vol_array = imageio.volread(dir_path, format='DICOM')
print(type(vol_array))   # >>> <class 'imageio.core.util.Array'>:  Numpy NDArray
# 이는 99개의 슬라이스(slice)로 구성된 512x512 크기의 3D 이미지임을 의미함
print(vol_array.shape)   # >>> (99, 512, 512)
# vol_array의 데이터 타입 출력: DICOM 이미지 파일에서 추출된 int16 데이터 타입
print(vol_array.dtype)   # >>> int16
print(vol_array[0])

print("#" * 50, 1)

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
for id in range(0, 99):
  fig.add_subplot(10, 10, id + 1)
  plt.imshow(vol_array[id])
plt.show()  # 전체 슬라이스 이미지들을 한 번에 표시

import torch
# numpy 배열을 PyTorch 텐서로 변환하고, float 타입으로 변환
vol = torch.from_numpy(vol_array).float()
# 첫 번째 차원에 채널 차원을 추가 (예: [C, D, H, W] 형태로 만듦)
vol = torch.unsqueeze(vol, 0)  # channel
# 두 번째 차원에 데이터 차원을 추가하여 [B, C, D, H, W] 형태로 만듦
vol = torch.unsqueeze(vol, 0)  # data size

print(vol.shape)  # >>> torch.Size([1, 1, 99, 512, 512])
  
print("#" * 50, 2)

mean = torch.mean(vol, dim=(3, 4), keepdim=True)  # mean over all of dim=(3, 4)
print(mean.shape)
std = torch.std(vol, dim=(3, 4), keepdim=True)    # std over all of dim=(3, 4)
print(std.shape)
vol = (vol - mean) / std
print(vol.shape)

print(vol[0, 0, 0])  

# B 고찰내용
1. DICOM 데이터의 읽기 및 numpy 배열 변환을 할 수 있다.
2. 데이터 형태 및 타입 확인할 수 있다.
3. 이미지 시각화를 할 수 있다.
4. numpy를 텐서로 변환할 수 있다.
5. 정규화를 할 수 있다.
6. 3D 볼륨 데이터에서 특정 슬라이스에 접근할 수 있다.

In [ ]:
import csv
import os
import numpy as np

wine_path = os.path.join(os.path.pardir, "_00_data", "d_tabular-wine", "winequality-white.csv")
# 데이터를 NumPy 배열로 로드 (헤더를 건너뛰고, ';'로 구분된 값)
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
print(wineq_numpy.dtype)
print(wineq_numpy.shape)
print(wineq_numpy)
print()
# CSV 파일에서 컬럼 목록 읽기
col_list = next(csv.reader(open(wine_path), delimiter=';'))
print(col_list)
print()

print("#" * 50, 1)

import torch
# NumPy 배열을 PyTorch 텐서로 변환
wineq = torch.from_numpy(wineq_numpy)
print(wineq.dtype)
print(wineq.shape)
print()

# 특성과 타겟을 분리
data = wineq[:, :-1]  # Selects all rows and all columns except the last
print(data.dtype)
print(data.shape)
print(data)
print()

target = wineq[:, -1]  # Selects all rows and the last column
print(target.dtype)
print(target.shape)
print(target)
print()

target = target.long()  # treat labels as an integer
print(target.dtype)
print(target.shape)
print(target)
print()

print("#" * 50, 2)

# 원-핫 인코딩을 위한 단위 행렬 생성
eye_matrix = torch.eye(10)
# We use the 'target' tensor as indices to extract the corresponding rows from the identity matrix
# It can generate the one-hot vectors for each element in the 'target' tensor
onehot_target = eye_matrix[target]

print(onehot_target.shape)  # >>> torch.Size([4898, 10])
print(onehot_target[0])
print(onehot_target[1])
print(onehot_target[-2])
print(onehot_target)

print("#" * 50, 3)
# 데이터 정규화
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
data = (data - data_mean) / torch.sqrt(data_var)
print(data)

print("#" * 50, 4)

from sklearn.model_selection import train_test_split
# 데이터 분할: 훈련 세트와 검증 세트
X_train, X_test, y_train, y_test = train_test_split(data, onehot_target, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


def get_wine_data():  #함수화
  # 데이터 로드 및 전처리 함수 정의
  wine_path = os.path.join(os.path.pardir, os.path.pardir, "_00_data", "d_tabular-wine", "winequality-white.csv")
  wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
  
  # NumPy 배열을 PyTorch 텐서로 변환
  wineq = torch.from_numpy(wineq_numpy)
  # 특성과 타겟을 분리
  data = wineq[:, :-1]  # Selects all rows and all columns except the last
  target = wineq[:, -1].long()  # treat labels as an integer
  # 원-핫 인코딩
  eye_matrix = torch.eye(10)
  onehot_target = eye_matrix[target]
  # 데이터 정규화
  data_mean = torch.mean(data, dim=0)
  data_var = torch.var(data, dim=0)
  data = (data - data_mean) / torch.sqrt(data_var)

  X_train, X_valid, y_train, y_valid = train_test_split(data, onehot_target, test_size=0.2)

  return X_train, X_valid, y_train, y_valid

# C 고찰내용
1. csv 파일을 읽고 로드할 수 있다.
2. 타겟을 정수형으로 변환하고 원 핫 인코딩 후 정규화할 수 있다.
* 원 핫 인코딩은 보통 범주형 데이터를 수치형으로 바꿀 때 사용한다.
3. 데이터분할이 가능하다.
4. 함수화하여 사용 가능하다.

In [ ]:
# https://medium.com/analytics-vidhya/implement-linear-regression-on-boston-housing-dataset-by-pytorch-c5d29546f938
# https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
import torch
from sklearn.datasets import fetch_california_housing
# California Housing 데이터셋을 로드합니다.
housing = fetch_california_housing()
print(housing.keys()) # 데이터셋의 키를 출력하여 어떤 정보가 포함되어 있는지 확인합니다.

# 데이터와 관련된 정보 출력
print(type(housing.data))  # 데이터 타입 확인 (일반적으로 <class 'numpy.ndarray'>)
print(housing.data.dtype)  # 데이터의 타입 확인 (예: float64)
print(housing.data.shape)  # 데이터 배열의 형태 확인 (예: (20640, 8))
print(housing.feature_names)  # 데이터의 특성 이름 출력

print(housing.target.shape)  # 타겟 배열의 형태 확인 (예: (20640,))
print(housing.target_names)  # 타겟의 이름 출력 (California Housing 데이터는 타겟 이름이 없음)

print("#" * 50, 1)

import numpy as np

print(housing.data.min(), housing.data.max())
#정규화
data_mean = np.mean(housing.data, axis=0)
data_var = np.var(housing.data, axis=0)
data = (housing.data - data_mean) / np.sqrt(data_var)
target = housing.target


print(data.min(), data.max()) # 정규화된 데이터 최소값과 최대값을 출력하여 정규화 확인.

print("#" * 50, 2)

from sklearn.model_selection import train_test_split
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# 훈련 데이터와 테스트 데이터의 형태를 출력하여 확인합니다.
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

# D 고찰내용
1. fetch_california_housing() 함수를 통해 California Housing 데이터셋을 로드할 수 있다.
2. 정규화의 필요성
* 데이터의 범위가 서로 다를 경우, 모델학습의 성능을 향상시키기 위해 정규화가 필요하다.
* 방법 : 평균과 분산계산 ->  정규화 적용 -> 정규화 확인
3. 훈련 및 테스트 데이터로 분할 한다.
* 모델의 일반화 성능 평가와 과적합 방지를 위해 분할한다. 
4. 훈련데이터와 테스트 데이터의 형태를 출력하여 검사할 필요가 있다.

In [ ]:
import os
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75) # PyTorch의 출력 옵션 설정: 배열의 끝부분을 2개 항목만 표시하고, 전체 출력 개수 제한

bikes_path = os.path.join(os.path.pardir, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

bikes_numpy = np.loadtxt(   # CSV 파일에서 데이터를 NumPy 배열로 로드
  fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
  converters={
    1: lambda x: float(x[8:10])  # 2011-01-07 --> 07 --> 7.0
  }
)
bikes = torch.from_numpy(bikes_numpy)
print(bikes.shape)

# 데이터를 하루 단위로 재구성 (730일, 24시간, 17개의 특성)
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
print(daily_bikes.shape)  # >>> torch.Size([730, 24, 17])
# 특성과 타겟을 분리
daily_bikes_data = daily_bikes[:, :, :-1]
daily_bikes_target = daily_bikes[:, :, -1].unsqueeze(dim=-1)

print(daily_bikes_data.shape)
print(daily_bikes_target.shape)

print("#" * 50, 1)
# 첫 번째 날의 데이터 추출
first_day_data = daily_bikes_data[0]
print(first_day_data.shape)

# 날씨 상황 열을 정수형으로 출력
print(first_day_data[:, 9].long())
# 날씨 상황을 원-핫 인코딩하기 위한 단위 행렬 생성
eye_matrix = torch.eye(4)
print(eye_matrix)

# 날씨 상황 열을 원-핫 인코딩
weather_onehot = eye_matrix[first_day_data[:, 9].long() - 1]
print(weather_onehot.shape)
print(weather_onehot)

# 원-핫 인코딩된 날씨 데이터를 기존의 데이터와 연결
first_day_data_torch = torch.cat(tensors=(first_day_data, weather_onehot), dim=1)
print(first_day_data_torch.shape)
print(first_day_data_torch)

print("#" * 50, 2)

# 모든 날에 대해 날씨 상황을 원-핫 인코딩하고, 기존 데이터와 연결
day_data_torch_list = []

for daily_idx in range(daily_bikes_data.shape[0]):  # range(730)
  day = daily_bikes_data[daily_idx]  # day.shape: [24, 16]
  weather_onehot = eye_matrix[day[:, 9].long() - 1]
  day_data_torch = torch.cat(tensors=(day, weather_onehot), dim=1)  # day_data_torch.shape: [24, 20]
  day_data_torch_list.append(day_data_torch)

print(len(day_data_torch_list))
daily_bikes_data = torch.stack(day_data_torch_list, dim=0)
print(daily_bikes_data.shape)

print("#" * 50, 3)
# 데이터에서 'instant'와 'wheathersit' 열을 제거하고 나머지 열을 사용
print(daily_bikes_data[:, :, :9].shape, daily_bikes_data[:, :, 10:].shape)
daily_bikes_data = torch.cat(
  [daily_bikes_data[:, :, 1:9], daily_bikes_data[:, :, 10:]], dim=2
) # Drop 'instant' and 'whethersit' columns
print(daily_bikes_data.shape)

# 온도 열을 정규화 (평균이 0, 표준편차가 1이 되도록 변환)
temperatures = daily_bikes_data[:, :, 8]
daily_bikes_data[:, :, 8] = (daily_bikes_data[:, :, 8] - torch.mean(temperatures)) / torch.std(temperatures)


# E 고찰 내용
1. 데이터 재구성
* bikes.view(-1, 24, bikes.shape[1])로 데이터셋을 하루단위로 재구성할 수 있다.
* 특성과 타겟을 분리하고 나머지 열을 특성으로 사용가능하다.
2. 원 핫 인코딩
* torch.eye(4)를 통해 단위 행렬을 생성할 수 있다.
* eye_matrix[first_day_data[:, 9].long() - 1]를 통해 날씨 상황 열을 원-핫 인코딩하여 범주형 변수를 수치형 벡터로 변환할 수 있다.
* torch.cat()로 기존 데이터와 원 핫 인코딩된 데이터를 합칠 수 있다.
3. 데이터 정리 및 처리
* 모든 날의 데이터를 리스트로 저장하고, torch.stack를 사용하여 텐서로 변환할 수 있다.
* torch.cat를 사용하여 불필요한 열을 제거할 수 있다.